This is a dev notebook of a solver for matter effect.

In [2]:
%matplotlib inline
%load_ext snakeviz

In [3]:
import numpy as np
from scipy.integrate import odeint
from scipy.integrate import ode
import matplotlib.pylab as plt

In [4]:
# import neuosc as no

### Expectations

Before any calculation, I have calculated the results using Mathematica. The system to be solved is

$$
i \partial_x 
$$

The parameters used before are (in units of $\mathrm{MeV}$ or $\mathrm{MeV}^2$):

$\theta_v = \theta_{13} = 0.153077$

$\delta m^2 = \delta m_{13}^2 = 2.6*10^{-15}$

$\omega_v=6.5*10^{-17}$

$\lambda_0 = 0.5 \lambda_{MSW} = 3.09888*10^{-17}$

$\omega_m = 3.66619*10^{-17}$

$\theta_m = 0.162129$

$k_1= 1; k_2 = 1/90$

$\{A_1,A_2\} = \{0.00003588645221954444, 0.06486364865874367\}$ in units of omegam


Using these parameters,

1. Only one frequency case the oscillation weavelength is of the order $\hat x = \omega_m x\sim 1000000$
2. IF we are going to see the FULL oscillation, we expect a calculation range of $\hat x \sim 10^8$.

In [5]:
# Parameters are shared by ALL methods in this notebook

endpoint = 10000; # integration range
dx = 10.0; # step size
lam0 = 0.845258; # in unit of omegam, omegam = 3.66619*10^-17
dellam = np.array([0.00003588645221954444, 0.06486364865874367]); # deltalambda/omegam
ks = [1.0,1.0/90]; # two k's
thm = 0.16212913985547778; # theta_m

savestep = 1;

# REAL SYSTEM

We can also make it real.

In [50]:
### Real System
psi40, x40 = [1.0, 0.0, 0.0, 0.0], 0 # initial condition

xlin4 = np.arange(dx,endpoint+1*dx, dx)

psi4 = np.zeros([len(xlin4)  , 4])


xlin4save = np.zeros(len(xlin4)/savestep);
psi4save = np.zeros([len(xlin4save)  , 5])



#########################
# Make the equation all Real
#########################

def hamiltonian(x, deltalambda, k, thetam):
    
#    return (-0.5 + 0.5 * deltalambda * np.sin(k*x) * np.cos(2*thetam) ) * no.pauli_matrices(3) - 0.5 * deltalambda * np.sin(k*x) * np.sin(2*thetam) * no.pauli_matrices(1)     # This hamiltonian is not working
#    return [[ 0,   0.5* np.sin(2*thetam) * deltalambda * np.sin(k*x) * np.exp( 1.0j * ( - x - np.cos(2*thetam) * (deltalambda * np.cos(k*x) / k)  ) )     ],   [ 0.5* np.sin(2*thetam) * deltalambda * np.sin(k*x) * np.exp( -1.0j * ( - x - np.cos(2*thetam) * ( deltalambda /k * np.cos(k*x) )  ) ), 0 ]]   # this is the Hamiltonian that I used in MMA exactly  
    return np.array( [[ 0,   0.5* np.sin(2*thetam) * ( deltalambda[0] * np.sin(k[0]*x) + deltalambda[1] * np.sin(k[1]*x) ) * np.exp( 1.0j * ( - x - np.cos(2*thetam) * (  ( deltalambda[0]/k[0] * np.cos(k[0]*x) + deltalambda[1]/k[1] * np.cos(k[1]*x) ) )  ) )     ],   [ 0.5* np.sin(2*thetam) * ( deltalambda[0] * np.sin(k[0]*x) + deltalambda[1] * np.sin(k[1]*x) ) * np.exp( -1.0j * ( - x - np.cos(2*thetam) * ( deltalambda[0] /k[0] * np.cos(k[0]*x) + deltalambda[1] /k[1] * np.cos(k[1]*x) )  ) ), 0 ]]  ) # Hamiltonian for double frequency


def hamiltonian4(x, deltalambda, k, thetam):
    
    hr = np.array(hamiltonian(x, deltalambda, k, thetam)).real;
    hi = np.array(hamiltonian(x, deltalambda, k, thetam)).imag;
    
    return np.array([[hi[0][0],hi[0][1],hr[0][0],hr[0][1]], [hi[1][0],hi[1][1],hr[1][0],hr[1][1]], [- hr[0][0], - hr[0][1], hi[0][0], hi[0][1]],  [- hr[1][0], - hr[1][1], hi[1][0], hi[1][1]] ] )

def sysdpsidt(x, psi, deltalambda, k, thetam):
    
    return np.dot(hamiltonian4(x, deltalambda, k, thetam), [psi[0], psi[1], psi[2], psi[3]])


In [51]:
hamiltonian4(10,dellam,ks,thm)

array([[ 0.        , -0.00023664,  0.        , -0.00111786],
       [ 0.00023664,  0.        , -0.00111786,  0.        ],
       [-0.        ,  0.00111786,  0.        , -0.00023664],
       [ 0.00111786, -0.        ,  0.00023664,  0.        ]])

In [52]:
xlin4[0]

10.0

In [53]:
## Real System

atol_req = 1e-8

sol4 = ode(sysdpsidt).set_integrator('dopri5', atol=atol_req)
sol4.set_initial_value(psi40, x40).set_f_params(dellam,ks,thm)

flag4 = 0
flag4save = 0

while sol4.successful() and sol4.t < endpoint:
    sol4.integrate(xlin4[flag4])
    if np.mod(flag4,savestep)==0:
        psi4save[flag4save] = [sol4.t, sol4.y[0],sol4.y[1],sol4.y[2],sol4.y[3]]
 
        with open(r"assets/ode-dopri5-range-"+str(endpoint)+"-step-"+str(dx)+"-atol-"+str(atol_req)+".csv", 'a') as f_handle:
            np.savetxt(f_handle, psi4save[flag4save])
        
        flag4save = flag4save + 1
    flag4 = flag4 + 1
    #   print sol.t, sol.y



In [36]:
prob0 = psi4save[:,0]**2+psi4save[:,2]**2
prob1 = psi4save[:,1]**2+psi4save[:,3]**2

#prob0_100=solodeint100[0][:,0]**2+solodeint100[0][:,2]**2
#prob1_100=solodeint100[0][:,1]**2+solodeint100[0][:,3]**2

In [41]:
#print prob0, prob1, prob0+prob1
np.save("assets/ode-dopri5-range-"+str(endpoint)+"-step-"+str(dx)+"-prob0",prob0)
np.save("assets/ode-dopri5-range-"+str(endpoint)+"-step-"+str(dx)+"-prob1",prob1)
np.save("assets/ode-dopri5-range-"+str(endpoint)+"-step-"+str(dx)+"-xlin4",xlin4)

print "assets/ode-dopri5-range-"+str(endpoint)+"-step-"+str(dx)+"-prob0"

assets/ode-dopri5-range-10000-step-10.0-prob0
